# Modelo

Iteración rápida con un modelo de U-Net

In [1]:
from sklearn.preprocessing import MinMaxScaler

from matplotlib import colors
from skimage import exposure
from tqdm.notebook import tqdm

# =================|
# Tensorflow
# =================

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, MaxPool2D, UpSampling2D, Concatenate
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import Sequence

from scipy import ndimage

#=========================

import rasterio as rio
import rasterio.plot as rio_plot
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import json
import typing as typ
import math
import time

# Semilla pseudoaleatoria
np.random.seed(24)

# Tamaño de las figuras
plt.rcParams["figure.figsize"] = (20,10)

# Ruta base de la carpeta de datos
DATA_BASE = "/home/ggonzr_cloud/deeplearn/data"

# Imagen
HEIGHT = 256
WIDTH = 256
CHANNELS = 4

## Definición del modelo

### Versión V2

Link: https://github.com/nikhilroxtomar/Multiclass-Segmentation-in-Unet/blob/master/model.py

In [2]:
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x

def build_unet(shape, num_classes):
    inputs = Input(shape)

    """ Encoder """
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

    return Model(inputs, output)

## Funciones de carga de datos

In [3]:
def create_img_array(num_img, heigth, width, channels: int = 1) -> typ.Tuple[np.array, np.array]:
    array_rsp = np.zeros((num_img, heigth, width, channels), dtype=np.float32)    
    return array_rsp

In [4]:
def load_channel_raster(path_raster_tiff: str, channel: int = 1) -> np.array:
    rsp = None
    with rio.open(path_raster_tiff, "r") as rf:
        rsp = rf.read(channel)
    return rsp 

In [5]:
def load_source_img(img_folder_path: str) -> np.array:
    # Obtener la referencia a las bandas
    # Un poco excesivo, pero por si las moscas, especificaremos el patron de cada imagen a mano
    aerosol_1 = glob.glob(f"{img_folder_path}/B01.*")[0]
    azul_2 = glob.glob(f"{img_folder_path}/B02.*")[0]
    verde_3 = glob.glob(f"{img_folder_path}/B03.*")[0]
    rojo_4 = glob.glob(f"{img_folder_path}/B04.*")[0]
    rojo_frontera_1_5 = glob.glob(f"{img_folder_path}/B05.*")[0]
    rojo_frontera_2_6 = glob.glob(f"{img_folder_path}/B06.*")[0]
    rojo_frontera_3_7 = glob.glob(f"{img_folder_path}/B07.*")[0]
    infrarrojo_8 = glob.glob(f"{img_folder_path}/B08.*")[0]
    infrarrojo_8A_9 = glob.glob(f"{img_folder_path}/B8A.*")[0]
    vapor_agua_9_10 = glob.glob(f"{img_folder_path}/B09.*")[0]
    onda_corta_1_11 = glob.glob(f"{img_folder_path}/B11.*")[0]
    onda_corta_2_12 = glob.glob(f"{img_folder_path}/B12.*")[0]
    
    # Cargar las cuatro bandas
    channels_list = [
        aerosol_1,
        azul_2,
        verde_3,
        rojo_4,
        rojo_frontera_1_5,
        rojo_frontera_2_6,
        rojo_frontera_3_7,
        infrarrojo_8,
        infrarrojo_8A_9,
        vapor_agua_9_10,
        onda_corta_1_11,
        onda_corta_2_12
    ]
    
    raster_bands = [
        load_channel_raster(r)
        for r in channels_list
    ]
    
    # Evitar los NaN, +Infinite, -Infinite
    raster_bands_clean = [
        np.nan_to_num(r, nan=0, posinf=0, neginf=0)
        for r in raster_bands
    ]
    
    # Normalizar los canales
    norm_data = lambda x: ((x - np.mean(x))/ np.std(x))
                           
    # Aplicar
    norm_raster_bands = [
        norm_data(raster_band)
        for raster_band in raster_bands_clean
    ]
            
    # Construir el arreglo y retornar
    return np.array(norm_raster_bands)

In [6]:
def load_mask_img(mask_folder_path: str) -> np.array:
    # Obtener la referencia de la máscara
    mask_path = glob.glob(f"{mask_folder_path}/labels.*")[0]
    
    # Retornar la máscara
    loaded_channel = load_channel_raster(mask_path)
    loaded_channel_clean = np.nan_to_num(loaded_channel, nan=0, posinf=0, neginf=0)
    
    return loaded_channel_clean

In [7]:
def ndvi(raster_array: np.array) -> np.array:    
    # Las dimensiones aca son [bandas, altura, ancho]
    # Formula: NDVI (Sentinel 2) = (B8 – B4) / (B8 + B4)
    # Se toma con base en la modificacion con 12 bandas    
    # Mayor informacion, por favor ver definicion del metodo: load_source_img
    red_channel = raster_array[3, :, :]
    infrared_channel = raster_array[7, :, :]
    
    # Evitar divisiones por cero e inestabilidades
    epsilon = 1e-8
    return ((infrared_channel - red_channel) / ((infrared_channel + red_channel) + epsilon))

In [8]:
def bsi(raster_array: np.array) -> np.array:    
    # Las dimensiones aca son [bandas, altura, ancho]    
    # Se toma con base en la modificacion con 12 bandas    
    # Mayor informacion, por favor ver definicion del metodo: load_source_img
    # BSI (Sentinel 2) = (B11 + B4) – (B8 + B2) / (B11 + B4) + (B8 + B2)
    
    swir_1_11 = raster_array[10, :, :]
    rojo_4 = raster_array[3, :, :]
    infrarrojo_8 = raster_array[7, :, :]
    azul_2 = raster_array[1, :, :]
    
    # Evitar divisiones por cero e inestabilidades
    epsilon = 1e-8
    
    # SWIR <-> ROJO
    swir_rojo = swir_1_11 + rojo_4
    # NIR <-> AZUL
    nir_azul = infrarrojo_8 + azul_2
    
    # Retornar
    return (swir_rojo - nir_azul) / (swir_rojo + nir_azul + epsilon)

In [9]:
def ndmi(raster_array: np.array) -> np.array:    
    # Las dimensiones aca son [bandas, altura, ancho]    
    # Se toma con base en la modificacion con 12 bandas    
    # Mayor informacion, por favor ver definicion del metodo: load_source_img
    # NDMI (Sentinel 2) = (B8 – B11) / (B8 + B11)
    swir_1_11 = raster_array[10, :, :]
    infrarrojo_8 = raster_array[7, :, :]
    
    # Evitar divisiones por cero e inestabilidades
    epsilon = 1e-8
    
    return (infrarrojo_8 - swir_1_11) / (infrarrojo_8 + swir_1_11)

In [10]:
def get_label_mask_path_folder(chip_id: str) -> str:
    # Retorna la ruta con base en el chip
    label_folder = f"{DATA_BASE}/ref_landcovernet_v1_labels/ref_landcovernet_v1_labels_{chip_id}"
    return label_folder

In [11]:
def label_mask_one_hot(raster_mask: np.array, classes: typ.List[int]) -> np.array:
    # Retorna una máscara One Hot por cada clase
    # Obtenerla en 2D solo con el primer canal
    raster_mask_2d = raster_mask 
    masks = [
        (raster_mask_2d == i).astype(np.uint16)
        for i in classes
    ]
    
    return np.array(masks)

## Secuenciadores

Para apoyarnos rápidamente en la definición del secuenciador, vamos a apoyarnos de las funciones anteriores

In [12]:
class IndexSequence(Sequence):
    # x_sample_path: Arreglo con el contenido de las rutas e ID
    # Administra los batch de trabajo.    
    def __init__(self, x_sample_path: np.array, batch_size: int = 32):
        self.x_sample_path = x_sample_path
        self.batch_size = batch_size
        self.classes = list(range(1,8))
    
    
    def __len__(self):
        return math.ceil(self.x_sample_path.shape[0] / self.batch_size)
    
    
    def load_img_index(self, source_path):
        # Cargar el raster inicial -> [canal, alto, ancho]
        source_raster = load_source_img(source_path)
        
        # Transformar la imagen
        ndvi_transformed = ndvi(source_raster)
        bsi_transformed = bsi(source_raster)
        ndmi_transformed = ndmi(source_raster)        
        
        # Empaquetar y retornar
        index_array = np.array([
            ndvi_transformed,
            bsi_transformed,
            ndmi_transformed
        ])
        
        return rio_plot.reshape_as_image(index_array)
    
    
    def load_img_mask(self, chip_id):
        # Cargar la máscara inicial -> [clase, alto, ancho]
        mask_folder_path = get_label_mask_path_folder(chip_id=chip_id)
        mask_raster = load_mask_img(mask_folder_path)
        
        # Sobre las clases
        mask_raster_classes = label_mask_one_hot(mask_raster, self.classes)        
        
        return rio_plot.reshape_as_image(mask_raster_classes)
                

    def __getitem__(self, idx):
        # Obtener el batch de datos para cargar
        batch_x = self.x_sample_path[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        
        # Arreglo de datos para imagenes
        source_images = np.array([
            self.load_img_index(row_data[2])
            for row_data in batch_x
        ])
        
        # Arreglo de datos con las máscaras
        mask_images = np.array([
            self.load_img_mask(row_data[1])
            for row_data in batch_x
        ])

        return source_images, mask_images

## Cargar los datos de las imagenes 

In [13]:
# Cargar JSON con las imagenes
images_df = pd.read_json(f"{DATA_BASE}/images_to_use.json")

In [15]:
# Determinar muestra
SAMPLE_SIZE = 0.25
sample_train = images_df.sample(frac=SAMPLE_SIZE, random_state=24)
sample_train_np = sample_train.to_numpy()

In [16]:
samples_number = sample_train_np.shape[0]
print(f"Muestras: {samples_number}")

Muestras: 7544


### Carga completa de las bandas

In [16]:
# Construir el arreglo
X = create_img_array (
    num_img = samples_number,
    heigth = HEIGHT,
    width = WIDTH,
    channels = 12
)

Y = create_img_array (
    num_img = samples_number,
    heigth = HEIGHT,
    width = WIDTH,  
    channels = 7
)

In [17]:
# Ejecutar la carga
for row_id in tqdm(range(len(sample_train_np))):
    row = sample_train_np[row_id]
    chip_id = row[1]
    source_path = row[2]
    
    # Cargar la imagen fuente
    source_raster = load_source_img(source_path)
    
    # Transformar con NDVI
    # Descartar para la version V2 de U-Net
    #source_raster = ndvi(source_raster)
    
    # Cargar la máscara
    mask_folder_path = get_label_mask_path_folder(chip_id=chip_id)
    mask_raster = load_mask_img(mask_folder_path)
    mask_raster_classes = label_mask_one_hot(mask_raster, list(range(1,8)))
    
    # A este punto tenemos dimensiones
    # Fuente (source) -> [canal, alto, ancho] -> [12, 256, 256]
    # Máscara -> [clase, alto, ancho] -> [7, 256, 256]
    
    # Aplicar transformaciones si son necesarias
    # Cambian el orden de las dimensiones
    # [alto, ancho, canal/clase] -> Por ejemplo: [256, 256, 12]
    
    # Almacenar    
    X[row_id] = rio_plot.reshape_as_image(source_raster)
    Y[row_id] = rio_plot.reshape_as_image(mask_raster_classes)
    break

  0%|          | 0/30178 [00:00<?, ?it/s]

## Conjuntos de entrenamiento, validación, test

### Por medio de arreglos

In [17]:
# Distribuir el conjunto de entrenamiento en dos, train y test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Distribuir el conjunto de entrenamiento en dos, train y valid (validation)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### Por medio de secuenciadores

In [17]:
# Distribuir la ruta de las imagenes en los conjuntos apropiados
X_seq_train, X_seq_test = train_test_split(sample_train_np, test_size=0.2, random_state=42)
# Validacion
X_seq_train, X_seq_val = train_test_split(X_seq_train, test_size=0.2, random_state=42)

In [23]:
# Guardar el conjunto de test
test_df = pd.DataFrame(X_seq_test)
test_df.columns = sample_train.columns
test_df

,image_id,chip_id,image_folder_path,cloud_mask_mean
0,33PWQ_13_20180504,33PWQ_13,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0
1,35NRD_15_20180113,35NRD_15,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0
2,31PGR_26_20181117,31PGR_26,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0
3,34LHJ_20_20180620,34LHJ_20,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0
4,32PPA_20_20180410,32PPA_20,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0
...,...,...,...,...
1504,34NBP_29_20180513,34NBP_29,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.000122
1505,33KXV_05_20180924,33KXV_05,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0
1506,34JBL_26_20180817,34JBL_26,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0
1507,36PZC_22_20180405,36PZC_22,/home/ggonzr_cloud/deeplearn/data/ref_landcove...,0.0


In [24]:
# Almacenar
test_df.to_json(f"{DATA_BASE}/test_set.json")

In [18]:
# Instanciar los secuenciadores: En este caso vamos a trabajar con indices
train_index_seq = IndexSequence(x_sample_path=X_seq_train)
val_index_seq = IndexSequence(x_sample_path=X_seq_val)

## Instanciar el modelo

### Versión V2

In [19]:
SOURCE_CHANNELS = 3
EPOCHS_TRAIN = 10
model_v2_shape = (HEIGHT, WIDTH, SOURCE_CHANNELS)
model_v2_classes = 7
model = build_unet(model_v2_shape, model_v2_classes)

model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=tf.keras.optimizers.Adam(1e-4))

2021-12-06 22:00:24.748442: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [20]:
callbacks = [
        ModelCheckpoint("../models/model_index.h5", verbose=1, save_best_model=True),
        ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-6),
        EarlyStopping(monitor="val_loss", patience=5, verbose=1)
]

In [20]:
# Entrenamiento con el arreglo
#results = model.fit(X_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,\
#                    validation_data=(X_valid, y_valid))

In [21]:
# Entrenamiento con secuenciadores
tic = time.perf_counter()
results = model.fit(
    x=train_index_seq,
    validation_data=val_index_seq,
    epochs=EPOCHS_TRAIN,
    callbacks=callbacks,
    use_multiprocessing=True
)

toc = time.perf_counter()
print(f"Tiempo del entrenamiento: {toc - tic:0.4f} seconds")

Epoch 1/10



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=24

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRI

  3/151 [..............................] - ETA: 17:19 - loss: 2.0043 - accuracy: 0.2012

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide


151/151 [==============================] - ETA: 0s - loss: 1.8452 - accuracy: 0.2593

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide



Epoch 00001: saving model to ../models/model_index.h5
151/151 [==============================] - 1113s 7s/step - loss: 1.8452 - accuracy: 0.2593 - val_loss: 1.8034 - val_accuracy: 0.2761 - lr: 1.0000e-04
Epoch 2/10
151/151 [==============================] - ETA: 0s - loss: 1.7369 - accuracy: 0.2724
Epoch 00002: saving model to ../models/model_index.h5
151/151 [==============================] - 402s 3s/step - loss: 1.7369 - accuracy: 0.2724 - val_loss: 1.7482 - val_accuracy: 0.2773 - lr: 1.0000e-04
Epoch 3/10
151/151 [==============================] - ETA: 0s - loss: 1.6829 - accuracy: 0.2820
Epoch 00003: saving model to ../models/model_index.h5
151/151 [==============================] - 401s 3s/step - loss: 1.6829 - accuracy: 0.2820 - val_loss: 1.6702 - val_accuracy: 0.2786 - lr: 1.0000e-04
Epoch 4/10
151/151 [==============================] - ETA: 0s - loss: 1.6420 - accuracy: 0.3170
Epoch 00004: saving model to ../models/model_index.h5
151/151 [==============================] - 399s

# Next section